# Procesamiento de Elecciones de Diputados Locales - Manzanillo (2012-2024)
## Objetivo
El propósito de este notebook es limpiar, estandarizar y estructurar los resultados electorales para las elecciones de Diputados Locales en Manzanillo, abarcando el periodo de 2012 a 2024. El objetivo final es generar archivos de datos limpios y consistentes que puedan ser fácilmente consolidados en una única base de datos para su posterior análisis y visualización geoespacial.

### Manzanillo Diputados locales 2012

In [11]:
import pandas as pd 

df = pd.read_csv("/Users/omartellez/Manzanillo/01_datos_brutos/Diputados_Locales/Manzanillo_DIP_LOCAL_2012.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   casilla_seccion  198 non-null    object
 1   distrito         198 non-null    int64 
 2   pan              198 non-null    int64 
 3   pri_na           198 non-null    int64 
 4   prd              198 non-null    int64 
 5   pt               198 non-null    int64 
 6   pvem             198 non-null    int64 
 7   mc               198 non-null    int64 
 8   adc              198 non-null    int64 
 9   voto_nulo        198 non-null    int64 
 10  voto_total       198 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 17.1+ KB


In [6]:
df.head()

,casilla_seccion,distrito,pan,pri_na,prd,pt,pvem,mc,adc,voto_nulo,voto_total
0,200 B1,13,138,182,14,30,7,10,5,24,410
1,201 B1,13,42,42,2,5,4,0,1,2,98
2,202 B1,13,161,143,16,16,11,3,0,18,368
3,202 C1,13,128,163,17,15,7,6,1,38,375
4,202 C2,13,136,153,17,9,7,1,0,24,347


In [12]:
import pandas as pd
import os

# --- BLOQUE DE DIAGNÓSTICO ---

# Ponemos la ruta base
BASE_DIR = '/Users/omartellez/Manzanillo' 

# Usamos el nombre de la carpeta en PLURAL, como lo indicaste
folder_path = os.path.join(BASE_DIR, '01_datos_brutos', 'Diputados_Locales')
file_path = os.path.join(folder_path, 'Manzanillo_DIP_LOCAL_2012.csv')

print(f"Intentando abrir el archivo en la ruta:\n{file_path}\n")

try:
    # Intentamos cargar el archivo
    df_test = pd.read_csv(file_path, encoding='latin-1')
    
    print("--- ¡ÉXITO! ARCHIVO CARGADO ---")
    print("El archivo fue encontrado y leído correctamente.\n")
    
    # Imprimimos la lista exacta de las columnas
    print("COLUMNAS REALES DETECTADAS:")
    print(df_test.columns.tolist())
    
    print("\nPRIMERAS 5 FILAS:")
    # Usamos display() si estás en un notebook para mejor formato
    try:
        display(df_test.head())
    except NameError:
        print(df_test.head())

except FileNotFoundError:
    print("--- ¡ERROR! ARCHIVO NO ENCONTRADO ---")
    print("Pandas no pudo encontrar el archivo en la ruta especificada.")
    print("Por favor, verifica si el nombre de la carpeta es exactamente 'Diputados_Locales' (plural).")
    print("Si se llama 'Diputado_Local' (singular), avísame para corregir la ruta en el script.")

except Exception as e:
    print(f"--- ¡OCURRIÓ OTRO ERROR! ---")
    print(e)

Intentando abrir el archivo en la ruta:
/Users/omartellez/Manzanillo/01_datos_brutos/Diputados_Locales/Manzanillo_DIP_LOCAL_2012.csv

--- ¡ÉXITO! ARCHIVO CARGADO ---
El archivo fue encontrado y leído correctamente.

COLUMNAS REALES DETECTADAS:
['ï»¿casilla_seccion', 'distrito', 'pan', 'pri_na', 'prd', 'pt', 'pvem', 'mc', 'adc', 'voto_nulo', 'voto_total']

PRIMERAS 5 FILAS:


,ï»¿casilla_seccion,distrito,pan,pri_na,prd,pt,pvem,mc,adc,voto_nulo,voto_total
0,200 B1Â,13,138,182,14,30,7,10,5,24,410
1,201 B1Â,13,42,42,2,5,4,0,1,2,98
2,202 B1Â,13,161,143,16,16,11,3,0,18,368
3,202 C1Â,13,128,163,17,15,7,6,1,38,375
4,202 C2Â,13,136,153,17,9,7,1,0,24,347


In [14]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

# CORRECCIÓN 1: Usamos el nombre de carpeta en PLURAL
input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Diputados_Locales')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Locales') # La salida puede ser singular, como acordamos

input_file = 'Manzanillo_DIP_LOCAL_2012.csv'
output_file = 'Manzanillo_DIP_LOCAL_2012_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
# CORRECCIÓN 2: Usamos la codificación 'utf-8-sig' para eliminar la marca BOM (ï»¿)
df = pd.read_csv(file_path, encoding='utf-8-sig')
print("Datos originales de Diputado Local 2012 cargados correctamente.")

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# La limpieza de espacios sigue siendo una buena práctica, la mantenemos
df_mapped.columns = df_mapped.columns.str.strip()

# a) Separar 'casilla_seccion' en 'seccion' y 'casilla'
# Esta línea ahora funcionará porque el nombre de la columna será leído correctamente
df_mapped[['seccion', 'casilla']] = df_mapped['casilla_seccion'].str.extract(r'(\d+)\s*(.*)', expand=True)
df_mapped['seccion'] = pd.to_numeric(df_mapped['seccion'], errors='coerce')
df_mapped = df_mapped.drop(columns=['casilla_seccion'])

# b) Eliminar columnas de contexto
if 'distrito' in df_mapped.columns:
    df_mapped = df_mapped.drop(columns=['distrito'])
    
# c) Asegurar que las columnas estándar existan
columnas_estandar = ['lista_nominal', 'voto_no_registrado']
for col in columnas_estandar:
    if col not in df_mapped.columns:
        df_mapped[col] = 0

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# (Esta sección no cambia)
competidores_2012 = ['pan', 'pri_na', 'prd', 'pt', 'pvem', 'mc', 'adc']
proxy_morena_2012 = ['prd', 'pt', 'mc']
oposicion_2012 = ['pan', 'pri_na', 'pvem', 'adc']
df_mapped['morena_coalitions'] = df_mapped[proxy_morena_2012].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2012].sum(axis=1)
df_mapped['voto_morena_solo'] = 0
df_mapped['voto_morena_aliados'] = df_mapped[proxy_morena_2012].sum(axis=1)
df_competidores = df_mapped[competidores_2012]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
# (Esta sección no cambia)
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri_na', 'prd', 'pt', 'pvem', 'mc', 'adc',
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
columnas_existentes = [col for col in columnas_finales if col in df_mapped.columns]
df_mapped = df_mapped[columnas_existentes]

print("\nDatos limpios de 2012 (Diputado Local) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2012 (Diputado Local) guardado en: {output_path}")

Datos originales de Diputado Local 2012 cargados correctamente.

Datos limpios de 2012 (Diputado Local) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  198 non-null    int64 
 1   casilla                  198 non-null    object
 2   lista_nominal            198 non-null    int64 
 3   pan                      198 non-null    int64 
 4   pri_na                   198 non-null    int64 
 5   prd                      198 non-null    int64 
 6   pt                       198 non-null    int64 
 7   pvem                     198 non-null    int64 
 8   mc                       198 non-null    int64 
 9   adc                      198 non-null    int64 
 10  morena_coalitions        198 non-null    int64 
 11  non_morena_coalitions    198 non-null    int64 
 12  voto_mor

### Manzanillo Diputados 2015

In [40]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Diputados_Locales')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Locales')

output_file = 'Manzanillo_DIP_LOCAL_2015_limpio.csv'

# --- 2. BUCLE PARA PROCESAR CADA DISTRITO ---
distritos = [11, 12, 13, 14] 
all_districts_dfs = []

COLUMNAS_FIJAS = ['seccion', 'casilla', 'lista_nominal', 'voto_no_registrado', 'voto_nulo', 'voto_total']

print(f"Iniciando procesamiento para Diputado Local 2015 para los distritos: {distritos}")

for dist in distritos:
    input_file = f'Manzanillo_DIP_LOCAL_2015_{dist}.csv'
    file_path = os.path.join(input_folder, input_file)
    
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig') # Usamos utf-8-sig para el BOM
        print(f"  -> Archivo encontrado y cargado: {input_file}")
    except FileNotFoundError:
        print(f"  -> Archivo NO encontrado, se omite: {input_file}")
        continue
        
    # --- 3. LIMPIEZA Y ESTRUCTURACIÓN (VERSIÓN DEFINITIVA) ---
    df_mapped = df.copy()
    
    # a) Renombrar columnas dañadas ('Unnamed: ...') ANTES de cualquier otra cosa
    if 'Unnamed: 16' in df_mapped.columns and 'Unnamed: 17' in df_mapped.columns:
        print(f"     Reparando encabezados dañados para Distrito {dist}...")
        df_mapped = df_mapped.rename(columns={'Unnamed: 16': 'voto_nulo', 'Unnamed: 17': 'voto_total'})

    # b) SOLUCIÓN DEFINITIVA: Limpieza robusta de TODOS los nombres de columna
    # Primero quitamos espacios al inicio/final, LUEGO pasamos a minúsculas.
    df_mapped.columns = df_mapped.columns.str.strip().str.lower()
    
    # c) Limpieza del resto de columnas
    df_mapped = df_mapped.rename(columns={'tipo_casilla': 'casilla'})
    col_distrito_a_eliminar = [col for col in df_mapped.columns if 'distrito' in col]
    df_mapped = df_mapped.drop(columns=col_distrito_a_eliminar)
    
    # --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS ---
    # Esta sección ahora funcionará correctamente para todos los archivos
    competidores_distrito = [col for col in df_mapped.columns if col not in COLUMNAS_FIJAS]
    partidos_morena_2015 = ['morena']
    oposicion_2015 = [col for col in competidores_distrito if col != 'morena']
    df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2015].sum(axis=1)
    df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2015].sum(axis=1)
    df_mapped['voto_morena_solo'] = df_mapped['morena']
    df_mapped['voto_morena_aliados'] = 0
    df_competidores = df_mapped[competidores_distrito]
    df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
    df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)
    
    if len(competidores_distrito) > 1:
        segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
        df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
        temp_df = df_competidores.copy()
        for index, row in temp_df.iterrows():
            winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
            if pd.notna(winner_col) and winner_col in temp_df.columns:
                temp_df.loc[index, winner_col] = np.nan
        df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
        df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']
    else:
        df_mapped['segundo_lugar_votos'] = 0
        df_mapped['segundo_lugar_partido'] = "N/A"
        df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos']

        # ... (código anterior del bucle) ...

    # --- BLOQUE DE INSPECCIÓN FINAL (AÑADIR ANTES DE .append()) ---
    print(f"--- Inspeccionando datos procesados para Distrito {dist} ---")
    if 'seccion' in df_mapped.columns:
        print("Primeros 5 valores de la columna 'seccion' procesada:")
        print(df_mapped['seccion'].head().to_string()) # .to_string() para mejor formato
    else:
        print("¡ALERTA MÁXIMA! La columna 'seccion' ya no existe en este punto del script.")
    print("-" * 55)
    # --- FIN DEL BLOQUE DE INSPECCIÓN ---
    
    # Esta es la línea original que ya tenías
    all_districts_dfs.append(df_mapped)

# ... (resto del script) ...
        
    all_districts_dfs.append(df_mapped)

# --- 5. CONSOLIDACIÓN Y GUARDADO FINAL ---
if not all_districts_dfs:
    print("\nNo se procesó ningún archivo.")
else:
    df_2015_final = pd.concat(all_districts_dfs, ignore_index=True).fillna(0)
    
    print("\n\n--- Proceso de Unión Finalizado ---")
    print(f"Se han consolidado los datos de {len(all_districts_dfs)} distritos.")
    print("Datos limpios de 2015 (Diputado Local) con el modelo de análisis completo:")
    df_2015_final.info()
    

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_file)
    df_2015_final.to_csv(output_path, index=False)
    print(f"\nArchivo limpio y consolidado de 2015 (Diputado Local) guardado en: {output_path}")

Iniciando procesamiento para Diputado Local 2015 para los distritos: [11, 12, 13, 14]
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2015_11.csv
--- Inspeccionando datos procesados para Distrito 11 ---
Primeros 5 valores de la columna 'seccion' procesada:
0    228
1    228
2    228
3    228
4    228
-------------------------------------------------------
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2015_12.csv
     Reparando encabezados dañados para Distrito 12...
--- Inspeccionando datos procesados para Distrito 12 ---
Primeros 5 valores de la columna 'seccion' procesada:
0    249
1    250
2    250
3    250
4    250
-------------------------------------------------------
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2015_13.csv
--- Inspeccionando datos procesados para Distrito 13 ---
Primeros 5 valores de la columna 'seccion' procesada:
0    200
1    201
2    202
3    202
4    202
-------------------------------------------------------
  -> Archivo encont

### Manzanillo Diputados 2018

In [41]:
input_file = "Manzanillo_DIP_LOCAL_2018.csv"
output_file = 'Manzanillo_DIP_LOCAL_2018_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   distrito            217 non-null    int64 
 1   seccion             217 non-null    int64 
 2   tipo_casilla        217 non-null    object
 3   pan                 217 non-null    int64 
 4   pri                 217 non-null    int64 
 5   prd                 217 non-null    int64 
 6   pt                  217 non-null    int64 
 7   pvem                217 non-null    int64 
 8   morena              217 non-null    int64 
 9   es                  217 non-null    int64 
 10  pan_prd             217 non-null    int64 
 11  pri_pvem            217 non-null    int64 
 12  morena_pt_es        217 non-null    int64 
 13  morena_pt           217 non-null    int64 
 14  pt_es               217 non-null    int64 
 15  mor_es              217 non-null    int64 
 16  mc      

,distrito,seccion,tipo_casilla,pan,pri,prd,pt,pvem,morena,es,...,morena_pt_es,morena_pt,pt_es,mor_es,mc,na,Independiente 4,voto_no_registrado,voto_nulo,voto_total
0,13,200,B,69,33,1,12,118,98,4,...,4,0,0,3,6,10,7,0,10,381
1,13,201,B,22,5,1,1,15,17,1,...,0,0,0,0,6,1,3,0,3,77
2,13,202,B,60,28,5,6,54,141,4,...,9,2,0,1,15,15,14,0,0,363
3,13,202,C,57,31,3,11,62,128,14,...,2,2,0,0,16,10,17,0,24,381
4,13,202,C,63,43,7,12,57,132,5,...,4,0,0,0,17,12,15,0,15,390


In [42]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar nombres de columnas
df_mapped = df_mapped.rename(columns={
    'tipo_casilla': 'casilla',
    'Independiente 4': 'independiente_4',
    'mor_es': 'morena_es' # Asumiendo que 'mor_es' es 'morena_es'
})

# b) Asegurar que 'lista_nominal' exista
if 'lista_nominal' not in df_mapped.columns:
    df_mapped['lista_nominal'] = 0

# c) Eliminar columnas de contexto
if 'distrito' in df_mapped.columns:
    df_mapped = df_mapped.drop(columns=['distrito'])

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2018
competidores_2018 = [
    'pan', 'pri', 'prd', 'pt', 'pvem', 'morena', 'es', 'mc', 'na',
    'pan_prd', 'pri_pvem',
    'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es', # Usamos el nombre corregido
    'independiente_4'
]
partidos_morena_2018_total = [
    'morena', 'pt', 'es', 'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es'
]
partidos_morena_2018_aliados = [
    'pt', 'es', 'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es'
]
oposicion_2018 = [
    'pan', 'pri', 'prd', 'pvem', 'mc', 'na',
    'pan_prd', 'pri_pvem', 'independiente_4'
]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2018_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2018].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2018_aliados].sum(axis=1)

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2018]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pt', 'pvem', 'morena', 'es', 'mc', 'na', 'pan_prd', 
    'pri_pvem', 'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es', 'independiente_4',
    'morena_coalitions', 'non_morena_coalitions',
    'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2018 (Diputado Local) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2018 (Diputado Local) guardado en: {output_path}")


Datos limpios de 2018 (Diputado Local) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  217 non-null    int64 
 1   casilla                  217 non-null    object
 2   lista_nominal            217 non-null    int64 
 3   pan                      217 non-null    int64 
 4   pri                      217 non-null    int64 
 5   prd                      217 non-null    int64 
 6   pt                       217 non-null    int64 
 7   pvem                     217 non-null    int64 
 8   morena                   217 non-null    int64 
 9   es                       217 non-null    int64 
 10  mc                       217 non-null    int64 
 11  na                       217 non-null    int64 
 12  pan_prd                  217 non-null    int64 
 13  pri_pvem           

### Manzanillo Ayuntamientos 2021

In [44]:
input_file = "Manzanillo_DIP_LOCAL_2021.csv"
output_file = 'Manzanillo_DIP_LOCAL_2021_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CIRCUNSCRIPCION           0 non-null      float64
 1   ID_ESTADO                 239 non-null    int64  
 2   NOMBRE_ESTADO             239 non-null    object 
 3   ID_DISTRITO_LOCAL         239 non-null    int64  
 4   CABECERA_DISTRITAL_LOCAL  239 non-null    object 
 5   ID_MUNICIPIO              239 non-null    int64  
 6   MUNICIPIO                 239 non-null    object 
 7   SECCION                   239 non-null    int64  
 8   casilla                   239 non-null    object 
 9   ID_CASILLA                239 non-null    int64  
 10  EXT_CONTIGUA              239 non-null    int64  
 11  pan                       239 non-null    int64  
 12  pri                       239 non-null    int64  
 13  prd                       239 non-null    int64

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO,MUNICIPIO,SECCION,casilla,ID_CASILLA,...,pri_prd,voto_valido,voto_no_registrado,voto_nulo,voto_total,lista_nominal,ESTATUS_ACTA,TRIBUNAL,OBSERVACIONES,RUTA_ACTA
0,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,200,B,1,...,0,339,6,7,352,709,ACTA CASILLA,NaN,NaN,NaN
1,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,B,1,...,0,270,10,7,287,605,ACTA CASILLA,NaN,NaN,NaN
2,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,1,...,0,292,3,10,305,605,GRUPO DE RECUENTO,NaN,NaN,NaN
3,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,2,...,0,287,1,13,301,604,ACTA CASILLA,NaN,NaN,NaN
4,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,3,...,0,290,8,5,303,604,ACTA CASILLA,NaN,NaN,NaN


In [45]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto y vacías
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito_local', 
    'cabecera_distrital_local', 'id_municipio', 'municipio', 'id_casilla', 
    'ext_contigua', 'voto_valido', 'estatus_acta', 'tribunal', 
    'observaciones', 'ruta_acta',
    'morena_na' # Columna vacía
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para Dip. Local 2021
competidores_2021 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'na', 'es', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd'
]
partidos_morena_2021 = ['morena']
oposicion_2021 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'es', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd'
]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2021].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2021].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = 0 # No hubo coalición de voto para Morena

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2021]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']


# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'na', 'es', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'morena_coalitions', 'non_morena_coalitions',
    'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2021 (Diputado Local) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2021 (Diputado Local) guardado en: {output_path}")



Datos limpios de 2021 (Diputado Local) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  239 non-null    int64 
 1   casilla                  239 non-null    object
 2   lista_nominal            239 non-null    int64 
 3   pan                      239 non-null    int64 
 4   pri                      239 non-null    int64 
 5   prd                      239 non-null    int64 
 6   pvem                     239 non-null    int64 
 7   pt                       239 non-null    int64 
 8   mc                       239 non-null    int64 
 9   morena                   239 non-null    int64 
 10  na                       239 non-null    int64 
 11  es                       239 non-null    int64 
 12  rsp                      239 non-null    int64 
 13  fxm                

### Manzanillo Ayuntamientos 2024

In [51]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Diputados_Locales')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Locales')

output_file = 'Manzanillo_DIP_LOCAL_2024_limpio.csv'

# --- 2. BUCLE PARA PROCESAR CADA DISTRITO ---
distritos = [11, 12, 13, 14] 
all_districts_dfs = []

COLUMNAS_FIJAS = ['seccion', 'casilla', 'lista_nominal', 'voto_no_registrado', 'voto_nulo', 'voto_total']

print(f"Iniciando procesamiento para Diputado Local 2024 para los distritos: {distritos}")

for dist in distritos:
    input_file = f'Manzanillo_DIP_LOCAL_2024_{dist}.csv'
    file_path = os.path.join(input_folder, input_file)
    
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        print(f"  -> Archivo encontrado y cargado: {input_file}")
    except FileNotFoundError:
        print(f"  -> Archivo NO encontrado, se omite: {input_file}")
        continue
        
    # --- 3. LIMPIEZA Y ESTRUCTURACIÓN (VERSIÓN CORREGIDA DEFINITIVA) ---
    df_mapped = df.copy()

    # a) Estandarizar todos los nombres de columna a minúsculas
    df_mapped.columns = df_mapped.columns.str.lower()
    
    # b) Eliminar columnas de contexto innecesarias
    columnas_a_eliminar = [
        'id_estado', 'id_distrito_local', 'municipio_local', 'voto_valido', 
        'num_boletas_recibidas', 'num_boletas_sobrantes'
    ]
    col_con_bom = [col for col in df_mapped.columns if 'ï»¿' in col]
    if col_con_bom:
        columnas_a_eliminar.extend(col_con_bom)
    df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

    # c) Separar 'seccion' y 'casilla' de forma segura
    columna_original_seccion = 'seccion' 
    df_mapped[['seccion_temp', 'casilla_temp']] = df_mapped[columna_original_seccion].str.extract(r'(\d+)\s*(.*)', expand=True)
    df_mapped = df_mapped.drop(columns=[columna_original_seccion])
    df_mapped = df_mapped.rename(columns={'seccion_temp': 'seccion', 'casilla_temp': 'casilla'})
    df_mapped['seccion'] = pd.to_numeric(df_mapped['seccion'], errors='coerce')

    # --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS ---
    competidores_distrito = [col for col in df_mapped.columns if col not in COLUMNAS_FIJAS]
    
    partidos_morena_2024_total = [col for col in ['morena', 'pvem', 'pt', 'pvem_pt_morena', 'pvem_morena', 'pvem_pt', 'pt_morena'] if col in df_mapped.columns]
    partidos_morena_2024_aliados = [col for col in ['pvem', 'pt', 'pvem_pt_morena', 'pvem_morena', 'pvem_pt', 'pt_morena'] if col in df_mapped.columns]
    oposicion_2024 = [col for col in competidores_distrito if col not in partidos_morena_2024_total]
    
    df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2024_total].sum(axis=1)
    df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2024].sum(axis=1)
    df_mapped['voto_morena_solo'] = df_mapped['morena']
    df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2024_aliados].sum(axis=1)

    df_competidores = df_mapped[competidores_distrito]
    df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
    df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)
    
    if len(competidores_distrito) > 1:
        segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
        df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
        temp_df = df_competidores.copy()
        for index, row in temp_df.iterrows():
            winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
            if pd.notna(winner_col) and winner_col in temp_df.columns:
                temp_df.loc[index, winner_col] = np.nan
        df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
        df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']
    else:
        df_mapped['segundo_lugar_votos'] = 0
        df_mapped['segundo_lugar_partido'] = "N/A"
        df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos']
        
    all_districts_dfs.append(df_mapped)

# --- 5. CONSOLIDACIÓN Y GUARDADO FINAL ---
if not all_districts_dfs:
    print("\nNo se procesó ningún archivo.")
else:
    df_2024_final = pd.concat(all_districts_dfs, ignore_index=True).fillna(0)
    
    print("\n\n--- Proceso de Unión Finalizado ---")
    print(f"Se han consolidado los datos de {len(all_districts_dfs)} distritos.")
    print("Datos limpios de 2024 (Diputado Local) con el modelo de análisis completo:")
    df_2024_final.info()

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_file)
    df_2024_final.to_csv(output_path, index=False)
    print(f"\nArchivo limpio y consolidado de 2024 (Diputado Local) guardado en: {output_path}")

Iniciando procesamiento para Diputado Local 2024 para los distritos: [11, 12, 13, 14]
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2024_11.csv
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2024_12.csv
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2024_13.csv
  -> Archivo encontrado y cargado: Manzanillo_DIP_LOCAL_2024_14.csv


--- Proceso de Unión Finalizado ---
Se han consolidado los datos de 4 distritos.
Datos limpios de 2024 (Diputado Local) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   lista_nominal            227 non-null    int64  
 1   pan                      227 non-null    int64  
 2   pri                      227 non-null    int64  
 3   prd                      227 non-null    int64  
 4   pvem                     227 non-null    int64  
 

## Base consolidada elecciones Diputados en Manzanillo 2012-2024

In [54]:
import pandas as pd
import os
import glob

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'

# Apuntamos a la carpeta de Diputados Locales limpios
input_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Locales') 
output_folder = os.path.join(BASE_DIR, '03_base_consolidada')
output_file = 'diputados_locales_consolidado_2012_2024.csv'

# --- 2. ENCONTRAR Y CARGAR TODOS LOS ARCHIVOS LIMPIOS ---
search_pattern = os.path.join(input_folder, '*_limpio.csv')
clean_files = sorted(glob.glob(search_pattern))

if not clean_files:
    print("No se encontraron archivos limpios de Diputado Local para consolidar.")
else:
    all_dfs = []
    print(f"Se encontraron {len(clean_files)} archivos para consolidar:")
    for file_path in clean_files:
        print(f"  -> Cargando: {os.path.basename(file_path)}")
        
        # Extraer el año del nombre del archivo (ej. '..._2012_limpio.csv')
        try:
            year = int(os.path.basename(file_path).split('_')[-2])
        except (ValueError, IndexError):
            year = None

        df = pd.read_csv(file_path)
        df['anio'] = year
        df['tipo_eleccion'] = 'Diputados_Locales'
        
        all_dfs.append(df)

    # --- 3. CONSOLIDAR, ORDENAR Y GUARDAR ---
    consolidated_df = pd.concat(all_dfs, ignore_index=True)
    
    # Lógica de ordenamiento automático y robusto
    # Define los grupos de columnas que irán al principio y al final
    columnas_inicio = [
        'seccion', 'casilla', 'anio', 'tipo_eleccion', 'lista_nominal',
        'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
        'segundo_lugar_votos', 'margen_victoria',
        'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados'
    ]
    columnas_fin = ['voto_no_registrado', 'voto_nulo', 'voto_total']
    
    # El resto de las columnas (votos crudos) se ordenarán alfabéticamente en medio
    columnas_votos_crudos = sorted([
        col for col in consolidated_df.columns 
        if col not in columnas_inicio and col not in columnas_fin
    ])
    
    # Construimos la lista final del orden
    orden_final = columnas_inicio + columnas_votos_crudos + columnas_fin
    
    # Aplicamos el orden y rellenamos los nulos
    consolidated_df_ordenado = consolidated_df[orden_final]
    consolidated_df_final = consolidated_df_ordenado.fillna(0)
        
    # Guardar la base de datos consolidada
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_file)
    consolidated_df_final.to_csv(output_path, index=False)
    
    print("\n¡Base de datos de Diputado Local consolidada y ordenada creada exitosamente!")
    print(f"Guardada en: {output_path}")
    print("\nResumen de la base consolidada final:")
    consolidated_df_final.info()

Se encontraron 5 archivos para consolidar:
  -> Cargando: Manzanillo_DIP_LOCAL_2012_limpio.csv
  -> Cargando: Manzanillo_DIP_LOCAL_2015_limpio.csv
  -> Cargando: Manzanillo_DIP_LOCAL_2018_limpio.csv
  -> Cargando: Manzanillo_DIP_LOCAL_2021_limpio.csv
  -> Cargando: Manzanillo_DIP_LOCAL_2024_limpio.csv

¡Base de datos de Diputado Local consolidada y ordenada creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/03_base_consolidada/diputados_locales_consolidado_2012_2024.csv

Resumen de la base consolidada final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  1313 non-null   int64  
 1   casilla                  1313 non-null   object 
 2   anio                     1313 non-null   int64  
 3   tipo_eleccion            1313 non-null   object 
 4   lista_nominal            1313 non-null